# Desafio - beAnalytic

Realize a extração das informações que conseguir da base de dados listada no website:

https://steamdb.info/sales/


Armazene estes dados no Google BigQuery

Em seguida exporte ou conecte esses dados em um Google Sheets.

- Você deve criar um repositório público em um GIT para compartilhar conosco;

- Compartilhar o Sheets final (o link precisa ser público);

- Lembrar de pôr no repositório os arquivos da automação;

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time
from google.cloud import bigquery
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "max-age=0",
    "Cookie": "__cf_bm=xBFkSuIK7VKqalrmxwdKX7sWJySoL3CJBNXH7rM_3GE-1730910244-1.0.1.1-CCTcUmfbnid8JX22TZkh3qn9fy.BXPgWhQlgwwgMvBMWe5fVDYWL9SBUguwagb9Ut9psKgtlf2OyB8jLDZN4Cg; cf_clearance=9FkaT.Ng3HwnsLBDViKG8r8KSm38JpGmJt5hwjH8Z74-1730910244-1.2.1.1-V.1OBsBiNMfdCjzsq10lyAFrg2KjnbRc8x_.g0dUD0t2V7lQeNwDlwjFB7MkLTOnDcrRWd8dTnf_Ox85qP3H2QqlNwgeEe7uw2Fg474jf_ATwk22dR8TtdTNFC6EO.mDMEiFtd9nec2y_j9KgkwkcKn9MrVVyyEYociqI3z3LWXrPct7eOWnlgtF3sgJ4ru9n4s2MXorV.smuQJXywmpcbn9YKSrIx9vsXCq0jFbOSlZ.o3b8zgJsAroKpirX2e5.ugqPsWAta9oqaJ55p.QHKJml7.ADS9E_fZh_xf51UtSeNwuuRgt4Th016E8CqkYfJJ5n5THlbAP6zeIMZC7B49._So9COVGEcOjgLNbmbIQmcPP1j_E1KLhz7tB2hAPLWqvthf8XPOVsmlKRjLzpTtZer8l6J.ROJCXDbwr18w",
    "Referer": "https://steamdb.info/sales?__cf_chl_tk=454VqbUvsHY_kh9Je.cZNywooOshwoxNGmhjnjMFGnM-1730910244-1.0.1.1-XQFEaxTyXpSp.RGzRnAIxxsZc8r.B4uhhZYTFwNkHmU",
    "Sec-Ch-Ua": "\"Google Chrome\";v=\"123\", \"Not:A-Brand\";v=\"8\", \"Chromium\";v=\"123\"",
    "Sec-Ch-Ua-Arch": "x86",
    "Sec-Ch-Ua-Bitness": "64",
    "Sec-Ch-Ua-Full-Version": "123.0.6312.122",
    "Sec-Ch-Ua-Full-Version-List": "\"Google Chrome\";v=\"123.0.6312.122\", \"Not:A-Brand\";v=\"8.0.0.0\", \"Chromium\";v=\"123.0.6312.122\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Model": "\"\"",
    "Sec-Ch-Ua-Platform": "Linux",
    "Sec-Ch-Ua-Platform-Version": "6.8.0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
from bs4 import BeautifulSoup

url = 'https://steamdb.info/sales/'

chrome_options = Options()

chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


driver.get(url)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'tbody')))

# Criar o arquivo CSV para armazenar os dados
csv_filename = 'steamdb_sales.csv'
csv_file = open(csv_filename, mode='w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)

csv_writer.writerow(['Link do jogo', 'Nome do Jogo', 'Preço com desconto', 'Desconto aplicado', 'Avaliação do jogo', 'Data de Lançamento', 'Data de Fim da promoção'])


def extract_tbody_data():

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tbody = soup.find('tbody')
    if tbody:
        rows = tbody.find_all('tr')
        for row in rows:
            cells = row.find_all('td')

            if len(cells) >= 7:
                # Link do jogo
                link_tag = cells[0].find('a')
                game_link = link_tag['href'] if link_tag else ''

                # Nome do Jogo
                game_info = cells[2].get_text(separator="|", strip=True).split('|')[0]

                # Desconto aplicado
                discount = cells[3].text.strip()

                # Preço com desconto
                price = cells[4].text.strip()

                # Avaliação do jogo
                rating = cells[5].text.strip()

                # Data de lançamento do jogo
                born_date = cells[6].text.strip()

                # Data de Fim da promoção
                end_date = cells[7].text.strip()

                # Escrever os dados no arquivo CSV
                csv_writer.writerow([game_link, game_info, price, discount, rating, born_date, end_date])
                print([game_link, game_info, price, discount, rating, born_date, end_date])  # Para verificar o que está sendo extraído

extract_tbody_data()

while True:
    try:
        next_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '.next'))
        )

        next_button.click()

        WebDriverWait(driver, 20).until(
            EC.staleness_of(next_button)
        )

        extract_tbody_data()

        time.sleep(3)

    except Exception as e:
        print("Fim da paginação ou erro:", e)
        break


csv_file.close()

driver.quit()

print(f"Dados salvos em {csv_filename}")


# Envio de dados para o BigQuery

In [32]:
import os
from google.cloud import bigquery
from dotenv import load_dotenv

load_dotenv()

# A variável GOOGLE_APPLICATION_CREDENTIALS será lida do .env
client = bigquery.Client()

dataset_id = 'steam_sales_dataset' 
table_id = 'steam_sales'  

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, autodetect=True
)

file_path = 'steamdb_sales.csv'

# Carregar os dados para o BigQuery
with open(file_path, "rb") as source_file:
    load_job = client.load_table_from_file(
        source_file, f"{dataset_id}.{table_id}", job_config=job_config
    )

load_job.result()
print("Dados carregados com sucesso!")


Dados carregados com sucesso!
